In [2]:
% store -r housing_prepared
% store -r housing_labels
% store -r housing_data_pd
housing_labels = housing_labels
housing_data_pd = housing_data_pd

In [3]:
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6


class CombinedAttributeAdder(BaseEstimator, TransformerMixin):

    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms__per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms__per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributeAdder()),
    ('srd_scalar', StandardScaler())
])
housing_num = housing_data_pd.drop('ocean_proximity', axis=1)
num_attr = list(housing_num)  # list(pandas DF) will create a list of the header names
cat_attr = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    # name, transformer, list of columns names or indexes
    ('num', numeric_pipeline, num_attr),
    ('cat', OneHotEncoder(), cat_attr)
])
housing_prepared = full_pipeline.fit_transform(housing_data_pd)

In [4]:
# Starting with a liner regression model
from sklearn.linear_model import LinearRegression

l_reg = LinearRegression()
l_reg.fit(housing_prepared, housing_labels)

LinearRegression()

In [6]:
# try on training set
some_data = housing_data_pd.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print(l_reg.predict(some_data_prepared))
print(list(some_labels))

[ 85657.90192014 305492.60737488 152056.46122456 186095.70946094
 244550.67966089]
[72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [7]:
# check the RMSE for the whole training set
from sklearn.metrics import mean_squared_error

housing_predictions = l_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68627.87390018745

#### Notice that the RMSE value is 68k, which is too  much.
#### To fix this we can:
- Select another model
- Choose better features
- reduce the constraints

In [8]:
# DecisionTreeRegressor Model
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

In [11]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

0.0


#### Another way to evaluate the model is to:
- Use train_test_split function to split the training set into a smaller training and validation set.
- Then train using the training and evaluate using the validation set.
#### We will use K-Fold cross validation feature. It will split the data into K folds, 1 for evaluation and the others for training..


In [27]:
from sklearn.model_selection import cross_val_score

# cross_val_score expects a utility function (greater is better) rather than a cost function (smaller is better). So, the scoring function is the opposite of MSE.
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)


def display_scores(scorez):
    print(f"Scores: {scorez}")
    print(f"Mean: {scorez.mean()}")
    print(f"STD: {scorez.std()}")


display_scores(tree_rmse_scores)

Scores: [73388.89081265 69526.11285471 68138.26875887 71055.70049379
 67970.894958   78134.04341471 71082.24273858 72822.61578241
 68580.25237388 72878.3665384 ]
Mean: 71357.73887259862
STD: 2959.2932306058656


In [28]:
from sklearn.model_selection import cross_val_score

# cross_val_score expects a utility function (greater is better) rather than a cost function (smaller is better). So, the scoring function is the opposite of MSE.
scores = cross_val_score(l_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
l_reg_rmse_scores = np.sqrt(-scores)

display_scores(l_reg_rmse_scores)

Scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
STD: 2880.3282098180644


In [29]:
from sklearn.ensemble import RandomForestRegressor

# Random forest works by training many decision tress with random set of features and average these predictions
rfr = RandomForestRegressor()
rfr.fit(housing_prepared, housing_labels)
scores = cross_val_score(rfr, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
rfr_rmse_scores = np.sqrt(-scores)

display_scores(rfr_rmse_scores)

Scores: [51310.81233402 49097.88334955 47020.82288695 51999.92045315
 47256.03916104 51551.74120999 52270.36551712 49808.97674182
 48481.92924755 53875.59320091]
Mean: 50267.40841020938
STD: 2171.2941242678603


#### Save your model, hyperparams, trained params, cross-validation score and maybe the predictions.

In [ ]:
import joblib

joblib.dump(rfr, "random_forest_regressor.pkl")  # to save
rfr_loaded = joblib.load("random_forest_regressor.pkl")

## Model Fine-Tuning
Fine-tuning is done by trying different combination of hyperparams.
The trial of different combinations of hyperparams can be done using "GridSearchCV". It will do cross-validation to evaluate the combinations.
You define the combinations to try and it will try it by n number of folds.

In [22]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
# 3 * 4 from the first dict +
# 2 * 3 from the second dict *
# 5 folds (cv = 5) = 90

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [25]:
print(grid_search.best_params_)
grid_search.best_estimator_

{'max_features': 8, 'n_estimators': 30}


RandomForestRegressor(max_features=8, n_estimators=30)

Since 8 and 30 are chosen (maximum from each side), we should try higher numbers

In [30]:
curves = grid_search.cv_results_
for mean_score, params in zip(curves["mean_test_score"], curves["params"]):
    print(np.sqrt(-mean_score), params)

64015.32794586548 {'max_features': 2, 'n_estimators': 3}
55385.81422939445 {'max_features': 2, 'n_estimators': 10}
52765.00695744444 {'max_features': 2, 'n_estimators': 30}
59989.17574933045 {'max_features': 4, 'n_estimators': 3}
52469.71776755092 {'max_features': 4, 'n_estimators': 10}
50459.4432569964 {'max_features': 4, 'n_estimators': 30}
59852.09591934263 {'max_features': 6, 'n_estimators': 3}
52141.87544436116 {'max_features': 6, 'n_estimators': 10}
50194.02290930418 {'max_features': 6, 'n_estimators': 30}
58729.79821140529 {'max_features': 8, 'n_estimators': 3}
52529.83307246577 {'max_features': 8, 'n_estimators': 10}
50184.93003465762 {'max_features': 8, 'n_estimators': 30}
62801.90889315715 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53698.268819461344 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59625.651046185296 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52472.341054040684 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

Compare the RMSE here after fine-tuning the model (50184 with {'max_features': 8, 'n_estimators': 30}) with the RMSE before without fine-tuning (50267)

Some data preparation steps can be treated as hyperparams. Gird search will do this automatically by adding some combined features, handling outliers, features selections, etc.

GridSearch is good when you have few number of hyperparams. If you have huge number of these params, RandomizedSearchCV is better to be used.

In [33]:
# To check the importance of each feature:
feature_importance = grid_search.best_estimator_.feature_importances_
print(feature_importance)

[6.42658245e-02 6.10336731e-02 4.28724320e-02 1.46903759e-02
 1.49597221e-02 1.46504244e-02 1.41665385e-02 3.72043355e-01
 4.09704522e-02 1.12899974e-01 5.67994003e-02 3.37101753e-03
 1.82703508e-01 9.46248953e-05 1.63451015e-03 2.84416839e-03]


In [36]:
# let's show the above scores with their names
extra_attribs = ["rooms_per_hold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attr + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importance, attributes), reverse=True)

KeyboardInterrupt: 